In [1]:
import openai

import os

import pandas as pd

import time

import requests

In [2]:
openai.api_key = '<YOUR API KEY>'

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):

  messages = [{"role": "user", "content": prompt}]

  response = openai.ChatCompletion.create(

  model=model,

  messages=messages,

  temperature=0,

  )

  return response.choices[0].message["content"]

In [4]:
df_test = pd.read_csv("test.csv")

In [5]:
df_test.head()

,id,attributes.HappyHour,attributes.Ambience,hours.Tuesday,postal_code,attributes.AgesAllowed,attributes.GoodForDancing,attributes.OutdoorSeating,hours.Saturday,attributes.Corkage,...,attributes.BestNights,attributes.AcceptsInsurance,attributes.RestaurantsDelivery,attributes.DietaryRestrictions,attributes.BusinessAcceptsBitcoin,address,attributes.GoodForKids,attributes.GoodForMeal,hours,review
0,0,NaN,"b""{'touristy': False, 'hipster': False, 'roman...",b'11:30-0:0',b'M5A 1T1',NaN,NaN,b'True',b'17:0-2:0',NaN,...,NaN,NaN,b'False',NaN,NaN,b'366 Queen Street E',b'False',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Tuesday': '11:30-0:0', 'Wednesday': '11:30-0...",Overall wonderful experience. \n\nThe owner Fa...
1,1,NaN,"b""{'romantic': False, 'intimate': False, 'clas...",b'12:0-21:0',b'M6R 2N2',NaN,NaN,b'True',b'14:0-21:0',NaN,...,NaN,NaN,b'False',NaN,NaN,b'414 Roncesvalles Avenue',b'True',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '12:0-21:0', 'Tuesday': '12:0-21:0'...","VIBE: Nieghbourhood hole. TV playing sports, p..."
2,2,NaN,"b""{'romantic': False, 'intimate': False, 'tour...",NaN,b'53704',NaN,NaN,b'False',b'11:30-21:0',NaN,...,NaN,NaN,NaN,NaN,NaN,b'2817 E Washington Ave',b'True',NaN,"{'Monday': '11:30-21:0', 'Wednesday': '11:30-2...","Viet House is very good. The food was fresh, ..."
3,3,NaN,"b""{'romantic': False, 'intimate': False, 'clas...",b'17:0-21:30',b'T2E 6Z3',NaN,NaN,b'False',b'17:0-22:0',NaN,...,NaN,NaN,b'False',NaN,NaN,"b'2323 32 Avenue NE, Suite 108'",b'True',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-21:30',...",This is a really good place. Not truly authent...
4,4,b'False',"b""{'romantic': False, 'intimate': False, 'tour...",b'11:0-21:0',b'85085',NaN,b'False',b'True',b'11:0-22:0',NaN,...,"b""{'monday': False, 'tuesday': False, 'friday'...",NaN,b'False',NaN,NaN,b'2470 W Happy Valley Rd',b'True',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",This place is great. Think fast food Italian s...


In [14]:
review = list(df_test["review"])

In [15]:
type(review)

list

In [18]:
review[1]

'VIBE: Nieghbourhood hole. TV playing sports, patrons usually sitting alone but chatting with one another. \\n\\nFOOD:\\nI tried the pad thai. It was plentiful but bad--slimy noodles sitting in a soup of watery sauce. Not only was it totally unappetizing, but it wasn\\\'t even pad thai. \\n\\nGiven that Vicky\\\'s is about a 1 minute walk from me, I wanted to give it another chance. Perhaps I should have expected total crud from a $6,99 deal served to me late at 9pm. Perhaps other dishes would be better!\\n\\nI dont even remember what my next order was, because it was so similar to the "pad thai". Slimy noodles sitting in watery sauce. Except it was $9.99 and I felt very ripped off.\\n\\nI hate to hate on a neighbourhood joint, so here are the positives: \\n-portions are large \\n-prices are cheap\\n-open late\\n-service is super fast and friendly\\n-if you like slimy noodles sitting in watery sauce--welcome to your own personal nirvana! \\n\\nMaybe try the fish and chips? Going here r

In [19]:
len(review)

10000

In [23]:
restaurant_types = [
    "american (traditional)", "mexican", "italian", "chinese",
    "american (new)", "japanese", "mediterranean", "canadian (new)",
    "thai", "asian fusion"
]

In [187]:
pred = []
for r in review:
    prompt = f"Review: {r} Based on the above review, determine which restaurant style most match this review, food styles only including [american (traditional), mexican, italian, chinese, american (new), japanese, mediterranean, canadian (new), thai, asian fusion], please just provide your one prediction and do not make any prediction which is out of the list"
    response = get_completion(prompt)
    pred.append(response)

In [243]:
prediction = []
for i, p in enumerate(pred):
    found = False
    for t in restaurant_types:
        if t in p.lower():
            prediction.append(t)
            found = True
            break
    if not found:
        prediction.append("not found:" + p)

In [195]:
dic = {"Id": [], "Predicted": []}
for i, pred in enumerate(prediction):
    dic["Id"].append(i)
    dic["Predicted"].append(pred)


In [197]:
dic_df = pd.DataFrame.from_dict(dic)
dic_df.to_csv("predicted gpt.csv", index=False)